# Exploring geocoder.db

This notebook demonstrates how to:
- Connect to the geocoder.db SQLite database
- List available tables
- Preview table data
- Plot geometries if available

---

## 1. Import Required Libraries

We will use sqlite3, pandas, geopandas, shapely, and matplotlib for database access and plotting.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from shapely import wkb
from shapely.geometry import shape
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

## 2. Connect to geocoder.db SQLite Database

We will connect to the geocoder.db file using sqlite3.

In [ ]:
db_path = '../geocoder.db'  # Adjust path if needed
conn = sqlite3.connect(db_path)
print(f'Connected to {db_path}')

## 3. List Available Tables

Query the SQLite master table to list all tables present in the database.

In [ ]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
tables = pd.read_sql_query(tables_query, conn)
print('Tables in geocoder.db:')
display(tables)

## 4. Preview Table Data

Select a table and display the first few rows using pandas for inspection.

In [ ]:
# Pick a table to preview (change as needed)
preview_table = tables.iloc[0]['name'] if not tables.empty else None
if preview_table:
    print(f'Previewing table: {preview_table}')
    df_preview = pd.read_sql_query(f'SELECT * FROM {preview_table} LIMIT 5;', conn)
    display(df_preview)
else:
    print('No tables found.')

## 5. Plot Geometries from a Selected Table

If the selected table contains geometry data (WKB), we will plot a few geometries using shapely and matplotlib.

In [ ]:
# Try to plot geometries from a table with a geometry column
geom_table = None
geom_col = None
for t in tables['name']:
    try:
        geom_check = pd.read_sql_query(f"PRAGMA table_info('{t}')", conn)
    except Exception as e:
        print(f"Skipping table {t}: {e}")
        continue
    if 'the_geom' in geom_check['name'].values:
        geom_table = t
        geom_col = 'the_geom'
        break
    elif 'geometry' in geom_check['name'].values:
        geom_table = t
        geom_col = 'geometry'
        break
if geom_table:
    print(f'Plotting geometries from table: {geom_table}')
    geom_df = pd.read_sql_query(f"SELECT * FROM {geom_table} LIMIT 10", conn)
    geoms = []
    for wkb_bytes in geom_df[geom_col]:
        if wkb_bytes:
            try:
                geoms.append(wkb.loads(wkb_bytes))
            except Exception:
                pass
    if geoms:
        fig, ax = plt.subplots(figsize=(8,8))
        for g in geoms:
            try:
                x, y = g.xy if hasattr(g, 'xy') else (g.exterior.xy[0], g.exterior.xy[1])
                ax.plot(x, y)
            except Exception:
                try:
                    g = g.buffer(0)
                    x, y = g.exterior.xy
                    ax.plot(x, y)
                except Exception:
                    pass
        ax.set_title(f"Sample Geometries from {geom_table}")
        plt.show()
    else:
        print('No valid geometries found to plot.')
else:
    print('No table with geometry column found.')